# Fully Convolutional Networks for Semantic Segmentation

Run in the docker image found in https://github.com/waspinator/deep-learning-explorer


In [ ]:
import os
import sys
import zipfile
import random
import math
import re
import time
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

sys.path.insert(0, '../libraries')
import semantic.coco
import semantic.config
import semantic.weights
import semantic.fcn
import semantic.evaluate
import semantic.utils

%matplotlib inline
%config IPCompleter.greedy=True

HOME_DIR = '/home/keras'
ROOT_DATA_DIR = os.path.join(HOME_DIR, "data")
WEIGHTS_DIR = os.path.join(HOME_DIR, "data/weights")

# Dataset

Your data should be in the [COCO style format](http://cocodataset.org/#download).

Organize the dataset using the following structure:

```
DATA_DIR
│
└───annotations
│   │   instances_<subset><year>.json
│   
└───<subset><year>
    │   image021.jpeg
    │   image022.jpeg
```

In [ ]:
with zipfile.ZipFile(os.path.join(ROOT_DATA_DIR, "shapes.zip"), "r") as zip_ref:
    zip_ref.extractall(ROOT_DATA_DIR)

DATA_DIR = os.path.join(ROOT_DATA_DIR, "shapes")
MODEL_DIR = os.path.join(DATA_DIR, "logs")

In [ ]:
dataset_train = semantic.coco.CocoDataset()
dataset_train.load_coco(DATA_DIR, subset="shapes_train", year="2018")
dataset_train.prepare()

dataset_validate = semantic.coco.CocoDataset()
dataset_validate.load_coco(DATA_DIR, subset="shapes_validate", year="2018")
dataset_validate.prepare()

dataset_test = semantic.coco.CocoDataset()
dataset_test.load_coco(DATA_DIR, subset="shapes_test", year="2018")
dataset_test.prepare()

print(dataset_train.class_weight)


# Configuration

In [ ]:
class ShapesFcnConfig(semantic.config.Config):
    """Configuration for training on the Shapes dataset.
    Derives from the base Config class and overrides values specific
    to the Shapes dataset.
    """
    NAME = "shapes_fcn"

    IMAGES_PER_GPU = 1
    DATASET_SAMPLES = len(dataset_train.image_ids)
     
    STEPS_PER_EPOCH = DATASET_SAMPLES / IMAGES_PER_GPU

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + square, circle, triangle

    IMAGE_MAX_DIM = 256
    STEPS_PER_EPOCH = 128
    VALIDATION_STEPS = int(STEPS_PER_EPOCH / 10)
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.003
    ENCODER = 'VGG16'
    
config = ShapesFcnConfig()
config.display()

# Load Model

In [ ]:
model = semantic.fcn.FCN(config=config, model_dir=MODEL_DIR)

## Load weights

In [ ]:
inititalize_weights_with = 'imagenet'  # imagenet or last

if inititalize_weights_with == 'imagenet':
    weights_path = semantic.weights.vgg16_imagenet_weights_path()
else:
    weights_path = model.find_last()[1]

semantic.utils.load_weights(model.keras_model.layers, weights_path, by_name=True)

## Train head

Train using only the final layer

In [ ]:
epochs=1

In [ ]:
model.train(dataset_train, dataset_validate,
            epochs,
            layers='head')

## Fine tune

Train more slowly than before but using more or all the layers. Starts from the previous epoch. For example if you set the train head part above to have 4 epochs and you want to fine tune for 2, set the epochs below to 6

In [ ]:
epochs=6

In [ ]:
model.train(dataset_train, dataset_validate,
            epochs,
            layers='all')

# Predict

Predict on a sample image

In [ ]:
test_image_id = np.random.choice(dataset_test.image_ids, 1)[0]
image = Image.open(dataset_test.image_reference(test_image_id))
plt.figure(figsize=(8,4))
plt.axis('off')
plt.imshow(image)

In [ ]:
prediction = model.predict(image)

In [ ]:
plt.figure(figsize=(8,4))
im = plt.imshow(prediction, interpolation='none')
values = np.unique(prediction.ravel())
colors = [ im.cmap(im.norm(value)) for value in values]
patches = [ matplotlib.patches.Patch(color=colors[i], label="{}".format(dataset_test.class_info[i]['name'])) for i in values ]
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize='x-large')
plt.axis('off')
plt.show()

# Evaluate

Get the Mean IoU for each class in the dataset

In [ ]:
semantic.evaluate.evaluate(model, dataset_test)

In [ ]:
coco_output = semantic.utils.result_to_coco(prediction, dataset_test.class_info, config.IMAGE_MAX_DIM, config.IMAGE_MAX_DIM)

In [ ]:
coco_output